In [52]:
import pandas as pd
import numpy as np

In [3]:
train_ds = pd.read_csv("data/full_version/train/train_dataset.csv")
val_ds = pd.read_csv("data/full_version/val/val_dataset.csv")

In [4]:
total_df = train_ds.append(val_ds)

In [5]:
len(total_df)

37344

In [6]:
total_df.groupby('sex').babyname.count()

sex
boy     15546
girl    21798
Name: babyname, dtype: int64

In [53]:
total_df['first_letter'] = total_df.babyname.map(lambda x: x[0])
total_df['first_two_letter'] = total_df.babyname.map(lambda x: x[:2])
total_df['last_letter'] = total_df.babyname.map(lambda x: x[-1])
total_df['last_two_letter'] = total_df.babyname.map(lambda x: x[-2:])

In [54]:
total_df.head()

,babyname,sex,first_letter,last_letter,last_two_letter,first_two_letter
0,Tiahna,girl,T,a,na,Ti
1,Alethia,girl,A,a,ia,Al
2,Apolline,girl,A,e,ne,Ap
3,Sorell,girl,S,l,ll,So
4,Luciole,girl,L,e,le,Lu


In [39]:
by_first_letter = total_df.groupby('first_letter').babyname.count().sort_values(ascending=False).reset_index().rename(columns={'babyname':'cnt'})

by_first_letter['ratio'] = ((by_first_letter.cnt / by_first_letter.cnt.sum()).map(lambda x: np.round(x, 3)))
by_sex = total_df.groupby(['sex', 'first_letter']).babyname.count().sort_values(ascending=False).reset_index().rename(columns={'babyname':'cnt'})

by_first_letter_girl = by_sex[lambda x: x.sex == 'girl']
by_first_letter_boy = by_sex[lambda x: x.sex == 'boy']

by_first_letter_girl['ratio'] = ((by_first_letter_girl.cnt / by_first_letter_girl.cnt.sum()).map(lambda x: np.round(x, 3)))
by_first_letter_boy['ratio'] = ((by_first_letter_boy.cnt / by_first_letter_boy.cnt.sum()).map(lambda x: np.round(x, 3)))

In [81]:
def aggregate_by(target_column):

    by_first_letter = total_df.groupby(target_column).babyname.count().sort_values(ascending=False).reset_index().rename(columns={'babyname':'cnt'})

    by_first_letter['ratio'] = ((by_first_letter.cnt / by_first_letter.cnt.sum()).map(lambda x: np.round(x, 3)))
    by_sex = total_df.groupby(['sex', target_column]).babyname.count().sort_values(ascending=False).reset_index().rename(columns={'babyname':'cnt'})

    by_first_letter_girl = by_sex[lambda x: x.sex == 'girl']
    by_first_letter_boy = by_sex[lambda x: x.sex == 'boy']

    by_first_letter_girl['ratio'] = ((by_first_letter_girl.cnt / by_first_letter_girl.cnt.sum()).map(lambda x: np.round(x, 3)))
    by_first_letter_boy['ratio'] = ((by_first_letter_boy.cnt / by_first_letter_boy.cnt.sum()).map(lambda x: np.round(x, 3)))
    
    print(by_first_letter.head())
    
    print(by_first_letter_girl.head())
    print(by_first_letter_boy.head())
    
    return by_first_letter, by_first_letter_girl, by_first_letter_boy
    
by_first_letter, by_first_letter_girl, by_first_letter_boy = aggregate_by('first_two_letter')

  first_two_letter   cnt  ratio
0               Ma  1454  0.039
1               Ka   914  0.024
2               Sh   864  0.023
3               Ca   856  0.023
4               Al   842  0.023
    sex first_two_letter  cnt  ratio
0  girl               Ma  907  0.042
1  girl               Ka  726  0.033
2  girl               Sh  715  0.033
3  girl               Ca  624  0.029
4  girl               Ch  600  0.028
    sex first_two_letter  cnt  ratio
5   boy               Ma  547  0.035
12  boy               Ha  368  0.024
15  boy               Al  336  0.022
20  boy               Da  305  0.020
22  boy               De  300  0.019


/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [82]:
girl_df = by_first_letter_girl.rename(columns={'ratio': 'girl_ratio'})[['first_two_letter', 'girl_ratio']]
boy_df = by_first_letter_boy.rename(columns={'ratio': 'boy_ratio'})[['first_two_letter', 'boy_ratio']]
merged_ratio = pd.merge(girl_df, boy_df, on='first_two_letter')
merged_ratio['difference'] = (merged_ratio.girl_ratio - merged_ratio.boy_ratio).map(lambda x: np.abs(x))
merged_ratio.sort_values(by='difference', ascending=False)

,first_two_letter,girl_ratio,boy_ratio,difference
2,Sh,0.033,0.010,0.023
1,Ka,0.033,0.012,0.021
4,Ch,0.028,0.013,0.015
27,Ha,0.009,0.024,0.015
3,Ca,0.029,0.015,0.014
55,Ba,0.005,0.017,0.012
63,Ga,0.004,0.014,0.010
9,Jo,0.018,0.010,0.008
7,La,0.020,0.013,0.007
31,Co,0.009,0.016,0.007


In [83]:
by_first_letter, by_first_letter_girl, by_first_letter_boy = aggregate_by('last_letter')

girl_df = by_first_letter_girl.rename(columns={'ratio': 'girl_ratio'})[['last_letter', 'girl_ratio']]
boy_df = by_first_letter_boy.rename(columns={'ratio': 'boy_ratio'})[['last_letter', 'boy_ratio']]
merged_ratio = pd.merge(girl_df, boy_df, on='last_letter')
merged_ratio['difference'] = (merged_ratio.girl_ratio - merged_ratio.boy_ratio).map(lambda x: np.abs(x))
merged_ratio.sort_values(by='difference', ascending=False)

  last_letter    cnt  ratio
0           a  10693  0.286
1           e   6471  0.173
2           n   4813  0.129
3           y   1992  0.053
4           s   1825  0.049
     sex last_letter    cnt  ratio
0   girl           a  10198  0.468
1   girl           e   4989  0.229
3   girl           n   1690  0.078
8   girl           y   1036  0.048
11  girl           i    927  0.043
   sex last_letter   cnt  ratio
2  boy           n  3123  0.201
4  boy           o  1519  0.098
5  boy           e  1482  0.095
6  boy           s  1448  0.093
7  boy           r  1123  0.072


/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,last_letter,girl_ratio,boy_ratio,difference
0,a,0.468,0.032,0.436
1,e,0.229,0.095,0.134
2,n,0.078,0.201,0.123
10,o,0.008,0.098,0.090
7,s,0.017,0.093,0.076
9,r,0.009,0.072,0.063
11,d,0.007,0.059,0.052
6,l,0.026,0.066,0.040
17,k,0.001,0.027,0.026
8,t,0.015,0.040,0.025


In [84]:
by_first_letter, by_first_letter_girl, by_first_letter_boy = aggregate_by('last_two_letter')

girl_df = by_first_letter_girl.rename(columns={'ratio': 'girl_ratio'})[['last_two_letter', 'girl_ratio']]
boy_df = by_first_letter_boy.rename(columns={'ratio': 'boy_ratio'})[['last_two_letter', 'boy_ratio']]
merged_ratio = pd.merge(girl_df, boy_df, on='last_two_letter')
merged_ratio['difference'] = (merged_ratio.girl_ratio - merged_ratio.boy_ratio).map(lambda x: np.abs(x))
merged_ratio.sort_values(by='difference', ascending=False)

  last_two_letter   cnt  ratio
0              na  2419  0.065
1              ne  1713  0.046
2              ia  1510  0.040
3              ie  1022  0.027
4              on  1000  0.027
    sex last_two_letter   cnt  ratio
0  girl              na  2375  0.109
1  girl              ne  1476  0.068
2  girl              ia  1472  0.068
3  girl              la   909  0.042
5  girl              ta   837  0.038
    sex last_two_letter  cnt  ratio
4   boy              on  890  0.057
8   boy              an  763  0.049
12  boy              er  558  0.036
16  boy              in  482  0.031
17  boy              us  476  0.031


/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,last_two_letter,girl_ratio,boy_ratio,difference
0,na,0.109,0.003,0.106
2,ia,0.068,0.002,0.066
1,ne,0.068,0.015,0.053
38,on,0.005,0.057,0.052
3,la,0.042,0.002,0.040
25,an,0.009,0.049,0.040
4,ta,0.038,0.002,0.036
6,ra,0.036,0.002,0.034
45,er,0.004,0.036,0.032
7,da,0.033,0.001,0.032


In [89]:
merged_ratio[lambda x: x.last_two_letter == 'an']

,last_two_letter,girl_ratio,boy_ratio,difference
25,an,0.009,0.049,0.04


In [77]:
by_first_letter, by_first_letter_girl, by_first_letter_boy = aggregate_by('last_letter')

  last_letter    cnt  ratio
0           a  10693  0.286
1           e   6471  0.173
2           n   4813  0.129
3           y   1992  0.053
4           s   1825  0.049
     sex last_letter    cnt  ratio
0   girl           a  10198  0.468
1   girl           e   4989  0.229
3   girl           n   1690  0.078
8   girl           y   1036  0.048
11  girl           i    927  0.043
   sex last_letter   cnt  ratio
2  boy           n  3123  0.201
4  boy           o  1519  0.098
5  boy           e  1482  0.095
6  boy           s  1448  0.093
7  boy           r  1123  0.072


/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/junkwhinger/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [16]:
total_df.groupby('last_letter').babyname.count().sort_values(ascending=False)

last_letter
a    10693
e     6471
n     4813
y     1992
s     1825
o     1684
l     1602
i     1438
h     1383
r     1321
d     1072
t      946
k      442
m      382
g      190
z      180
u      169
f      127
c      122
v      107
b       93
w       90
x       59
p       58
j       44
q       24
-       12
.        2
J        1
C        1
,        1
Name: babyname, dtype: int64

In [19]:
total_df.groupby(['sex', 'last_letter']).babyname.count().sort_values(ascending=False)

sex   last_letter
girl  a              10198
      e               4989
boy   n               3123
girl  n               1690
boy   o               1519
      e               1482
      s               1448
      r               1123
girl  y               1036
boy   l               1031
      y                956
girl  i                927
boy   d                915
girl  h                883
boy   t                629
girl  l                571
boy   i                511
      h                500
      a                495
      k                422
girl  s                377
boy   m                338
girl  t                317
      r                198
      o                165
      d                157
boy   z                138
      g                137
      f                120
      u                119
      c                114
      v                101
      w                 87
      b                 78
      p                 54
girl  g                 53
      u   

In [29]:
by_last_two_df = total_df.groupby(['last_two_letter', 'sex']).babyname.count()

In [88]:
total_df[lambda x: x.sex == 'boy'][lambda x: x.last_two_letter == 'us'].sort_values(by='babyname')

,babyname,sex,first_letter,last_letter,last_two_letter,first_two_letter
30593,Abdus,boy,A,s,us,Ab
1828,Achilleus,boy,A,s,us,Ac
11101,Achillius,boy,A,s,us,Ac
10896,Adelfus,boy,A,s,us,Ad
10444,Adelphus,boy,A,s,us,Ad
9193,Adolphus,boy,A,s,us,Ad
4413,Adulfus,boy,A,s,us,Ad
24235,Aegeus,boy,A,s,us,Ae
5186,Aemilianus,boy,A,s,us,Ae
16151,Aemilius,boy,A,s,us,Ae
